In [149]:
import pandas as pd
import nltk
import numpy as np
from torchtext.data import Field
import csv
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

In [150]:
tokenize = lambda x: x.split()
TEXT = Field(sequential=True, tokenize=tokenize, lower=True, fix_length = 100)
LABEL = Field(sequential=False, use_vocab=False)

In [151]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [152]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [153]:
def get_body(n, df):
    return df.loc[lambda x: x["Body ID"] == n, "articleBody"].item()

In [8]:
with open('train_data_consolidated.csv', 'w') as f, open('val_data_consolidated.csv', 'w') as g:
    writer_train = csv.writer(f)
    writer_val = csv.writer(g)
    writer_train.writerow(['Body ID', 'Headline', 'Body', 'unrelated', 'related'])
    writer_val.writerow(['Body ID', 'Headline', 'Body', 'unrelated', 'related'])
    i = 1
    for index, row in train_stances.iterrows():
        id = row["Body ID"]
        body = get_body(id, train_bodies)
        headline = row["Headline"]
        stance = row["Stance"]
        if i % 4 == 0:
            writer_val.writerow([id, headline, body, 1 if stance == 'unrelated' else 0, 0 if stance == 'unrelated' else 1])
        else:
            writer_train.writerow([id, headline, body, 1 if stance == 'unrelated' else 0, 0 if stance == 'unrelated' else 1])
        i+=1

In [154]:
train = pd.read_csv("train_data_consolidated.csv")
print(train.shape)
train.head(100)

(37479, 5)


,Body ID,Headline,Body,unrelated,related
0,712,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,1,0
1,158,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,0,1
2,137,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,1,0
3,1923,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",0,1
4,154,'Nasa Confirms Earth Will Experience 6 Days of...,Thousands of people have been duped by a fake ...,0,1
5,962,Accused Boston Marathon Bomber Severely Injure...,A British fighter who travelled to Iraq to sto...,1,0
6,1739,Banksy 'Arrested & Real Identity Revealed' Is ...,If you’ve seen a story floating around on your...,0,1
7,882,British Aid Worker Confirmed Murdered By ISIS,The British Islamic State militant who has fea...,1,0
8,2327,Gateway Pundit,A British rapper whose father is awaiting tria...,0,1
9,1003,Kidnapped Nigerian schoolgirls: Government cla...,No one has died more times than Fidel Castro.\...,1,0


In [155]:
val = pd.read_csv("val_data_consolidated.csv")
print(val.shape)
val.head(50)

(12493, 5)


,Body ID,Headline,Body,unrelated,related
0,1034,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,1,0
1,2033,Identity of ISIS terrorist known as 'Jihadi Jo...,"Adding to Apple's iOS 8 launch troubles, a rep...",1,0
2,1468,Woman detained in Lebanon is not al-Baghdadi's...,An Iraqi official denied that a woman detained...,0,1
3,615,Vandals add rude paint job to $2.5m Bugatti (b...,"""Eh-oh!"" A 19-year-old student is claiming tha...",1,0
4,608,ISIL Beheads American Photojournalist in Iraq,"James Foley, an American journalist who went m...",0,1
5,1196,Would you take a bite out of the world's oldes...,Claim: An illustration created in response to ...,1,0
6,1658,Christian Bale Exits Steve Jobs Movie (Exclusive),The Islamic State released a video of its jiha...,1,0
7,1896,Insurgents killed in Nigeria despite alleged t...,Boko Haram has reportedly agreed to a cease-fi...,0,1
8,394,Report: ISIS Leader Abu Bakr Al-Baghdadi Assas...,An unverified photo claims to show the body of...,0,1
9,1337,Taylor Lianne Chandler: Michael Phelps' Cougar...,High quality global journalism requires invest...,1,0


In [156]:
train_datafields = [("Body ID", None),
                 ("Headline", TEXT),
                    ("Body", TEXT),
                 ("unrelated", LABEL),
                ("related", LABEL)]

trn, vld = TabularDataset.splits(
               path="",
               train='train_data_consolidated.csv', validation="val_data_consolidated.csv",
               format='csv',
               skip_header=True,
               fields=train_datafields)


In [159]:
print(len(trn[1].Headline))
print(trn[0].Headline)
print(len(trn[1].Body))
print(trn[0].Body)


11
['police', 'find', 'mass', 'graves', 'with', 'at', 'least', "'15", "bodies'", 'near', 'mexico', 'town', 'where', '43', 'students', 'disappeared', 'after', 'police', 'clash']
429
['danny', 'boyle', 'is', 'directing', 'the', 'untitled', 'film', 'seth', 'rogen', 'is', 'being', 'eyed', 'to', 'play', 'apple', 'co-founder', 'steve', 'wozniak', 'in', 'sony’s', 'steve', 'jobs', 'biopic.', 'danny', 'boyle', 'is', 'directing', 'the', 'untitled', 'film,', 'based', 'on', 'walter', "isaacson's", 'book', 'and', 'adapted', 'by', 'aaron', 'sorkin,', 'which', 'is', 'one', 'of', 'the', 'most', 'anticipated', 'biopics', 'in', 'recent', 'years.', 'negotiations', 'have', 'not', 'yet', 'begun,', 'and', 'it’s', 'not', 'even', 'clear', 'if', 'rogen', 'has', 'an', 'official', 'offer,', 'but', 'the', 'producers', '—', 'scott', 'rudin,', 'guymon', 'casady', 'and', 'mark', 'gordon', '—', 'have', 'set', 'their', 'sights', 'on', 'the', 'talent', 'and', 'are', 'in', 'talks.', 'of', 'course,', 'this', 'may', 'all'

In [160]:
TEXT.build_vocab(trn, vectors = 'glove.6B.100d')

In [101]:
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable

# class CNN_Text(nn.Module):
    
#     def __init__(self, vocab_size, emb_dim, output_dim, vocab, num_filters, kernel_size):
#         super(CNN_Text, self).__init__()
#         self.vocab_size = vocab_size
#         self.emb_dim = emb_dim
#         self.embed = nn.Embedding(vocab_size, self.emb_dim)
#         self.embed.weight.data.copy_(vocab.vectors)
#         self.embed.weight.requires_grad = False
        
#         self.conv = nn.Conv2d(in_channels = self.emb_dim, out_channels = num_filters, kernel_size = (kernel_size, 1), stride = (1,1), bias = True)
#         self.relu = nn.ReLU()
#         self.kernel_size = kernel_size
#         self.dropout = nn.Dropout(0.50)
#         self.num_filters = num_filters
#         self.output_dim = output_dim
        
        
#         #output layer
#         self.linear = nn.Linear(num_filters, self.output_dim)
#         self.sigmoid = nn.Sigmoid()
        
#     def forward_once(self, inputs):
#         max_seq = inputs.shape[0]
#         maxpool = nn.MaxPool3d(kernel_size = (1, max_seq - self.kernel_size + 1, 1), stride = (1,1,1))
        
#         inputs = inputs.transpose(0, 1) # batch x seq
#         #print(inputs.shape)
#         emb = self.embed(inputs) # batch x seq x emb_dim
#         # print(emb.shape)
#         emb = emb.transpose(1, 2) # batch x emb_dim x seq
#         # print(emb.shape)
#         emb = emb.unsqueeze(-1) # batch x emb_dim x seq x 1
#         #print(emb.shape)
        
#         conv1 = self.conv(emb) # batch x out_channel x (seq + 1 - kernel)
#         #print(conv1.shape)
#         relu1 = self.relu(conv1)
#         pool = maxpool(relu1) # batch x out_channel x 1 x 1
#         #print(pool.shape)
#         pool = pool.view(-1, self.num_filters) # batch x out_channel  
#         #print(pool.shape)
#         out = self.dropout(pool)
#         score = self.linear(out)
#         #print(score.shape)
#         pred = self.sigmoid(score)
        
#         return pred
    
#     def forward(self, input1, input2):
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)
#         print(output1.shape)
#         print(output2.shape)
#         return output1, output2

In [315]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class CNN_Text(nn.Module):
    
    def __init__(self, vocab_size, emb_dim, output_dim, vocab, num_filters, kernel_size):
        super(CNN_Text, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.embed = nn.Embedding(vocab_size, self.emb_dim)
        self.embed.weight.data.copy_(vocab.vectors)
        self.embed.weight.requires_grad = False
        
        self.conv13 = nn.Conv2d(in_channels = self.emb_dim, out_channels = num_filters, kernel_size = (3, 1), stride = (1,1), bias = True)
        self.conv14 = nn.Conv2d(in_channels = self.emb_dim, out_channels = num_filters , kernel_size = (4, 1), stride = (1,1), bias = True)
        self.conv15 = nn.Conv2d(in_channels = self.emb_dim, out_channels = num_filters, kernel_size = (5, 1), stride = (1,1),  bias = True)
        self.output_dim = output_dim
        self.dropout = nn.Dropout(0.25)
        self.num_filters = num_filters
       
        #output layer
        self.linear = nn.Linear(num_filters*3, self.output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def conv_and_pool(self, x, conv, kernel_size): #input should be batch x emb_dim x seq x 1
        #print("conv and poool")
        max_seq = x.shape[2]
        #print("max seq = {}".format(max_seq))
        x = F.relu(conv(x))  # batch x out_channel x (seq + 1 -  kernel) x 1 -> [128, 64, 98, 1])
        #print(x.shape)
        max_pool = nn.MaxPool3d(kernel_size = (1, max_seq - kernel_size + 1, 1), stride = (1,1,1))
        #print(x.shape)
        x = max_pool(x)
        #print("shape after max pooling: {}".format(x.shape)) # [128, 64, 1, 1]
        return x
        
    def forward_once(self, inputs):
        #max_seq = inputs.shape[0]
        #maxpool = nn.MaxPool3d(kernel_size = (1, max_seq - self.kernel_size + 1, 1), stride = (1,1,1))
        inputs = inputs.transpose(0, 1) # batch x seq
        emb = self.embed(inputs) # batch x seq x emb_dim
        emb = emb.transpose(1, 2) # batch x emb_dim x seq
        emb = emb.unsqueeze(-1) # batch x emb_dim x seq x 1
        
        res13 = self.conv_and_pool(emb, self.conv13, 3)
        res14 = self.conv_and_pool(emb, self.conv14, 4)
        res15 = self.conv_and_pool(emb, self.conv15, 5)
        
        res13 = res13.view(-1, self.num_filters)
        res14 = res14.view(-1, self.num_filters)
        res15 = res15.view(-1, self.num_filters)
        
        x = torch.cat((res13, res14, res15), 1)
        #print(x.shape)
        x = self.dropout(x)
        #print("DONE WITH CREATING CONVS")
        
        
        preds = self.linear(x)
        #print(preds.shape) # torch.Size([128, 2])
        
        preds = self.sigmoid(preds)
        
        return preds
    
    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [316]:
import torch
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))

        print("loss contrastive")
        print(loss_contrastive)
        return loss_contrastive

In [317]:
model_CNN = CNN_Text(len(TEXT.vocab), 100, 1, TEXT.vocab, 64, 5)
model_CNN.train()
import torch.optim as optim

optimizer = optim.Adam(model_CNN.parameters(), lr=1e-3)
criterion = ContrastiveLoss(margin=1.0)

In [318]:
train_iter, val_iter = BucketIterator.splits(
 (trn, vld),
 batch_sizes=(128,128),
 sort_key=lambda x: len(x.Body), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,
 repeat=False
)

In [319]:
import torch
class BatchGenerator:
    def __init__(self, dl, x_field1, x_field2, y_field):
        self.dl, self.x_field1, self.x_field2, self.y_field = dl, x_field1, x_field2, y_field
        
    def __len__(self):
        return len(self.dl)
    
    def __iter__(self):
        for batch in self.dl:
            X1 = getattr(batch, self.x_field1)
            X2 = getattr(batch, self.x_field2)
            y = getattr(batch, self.y_field)
            yield (X1, X2, y)
            
train_batch_it = BatchGenerator(train_iter, 'Body','Headline', 'unrelated')
val_batch_it = BatchGenerator(val_iter, 'Body', 'Headline', 'unrelated')

In [320]:
counter = []
loss_history = [] 
iteration_number= 0

In [321]:
def evaluate(model, iterator, criterion):
    counter = []
    loss_history = [] 
    iteration_number= 0
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(iterator,0):
            img0, img1 , label = data
            img0, img1 , label = img0, img1 , label
            output1,output2 = model(img0,img1)
            loss_contrastive = criterion(output1,output2,label.float())
            if i %10 == 0 :
                print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
                iteration_number +=10
                counter.append(iteration_number)
                loss_history.append(loss_contrastive.item())

In [322]:
# Train the model
for epoch in range(0,2):
    for i, data in enumerate(train_batch_it,0):
        img0, img1 , label = data
        img0, img1 , label = img0, img1 , label
        optimizer.zero_grad()
        output1,output2 = model_CNN(img0,img1)
        loss_contrastive = criterion(output1,output2,label.float())
        loss_contrastive.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())


loss contrastive
tensor(0.6683, grad_fn=<MeanBackward1>)
Epoch number 0
 Current loss 0.6682829260826111

loss contrastive
tensor(0.6698, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6030, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6540, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6283, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6004, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6335, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6559, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6440, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.5312, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.5749, grad_fn=<MeanBackward1>)
Epoch number 0
 Current loss 0.5748898983001709

loss contrastive
tensor(0.5253, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.6331, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.4481, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.5193, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.5424, grad_fn=<MeanBa

loss contrastive
tensor(0.1719, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1965, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2006, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1959, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1724, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1805, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1829, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1702, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1921, grad_fn=<MeanBackward1>)
Epoch number 0
 Current loss 0.19211864471435547

loss contrastive
tensor(0.1635, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1995, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2008, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1599, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2029, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2101, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2148, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1934, grad_f

loss contrastive
tensor(0.1907, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1377, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2047, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1464, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1704, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1645, grad_fn=<MeanBackward1>)
Epoch number 0
 Current loss 0.16445112228393555

loss contrastive
tensor(0.1618, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1958, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1758, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1359, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1556, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1754, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2215, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.2006, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1850, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1766, grad_fn=<MeanBackward1>)
Epoch number 0
 Current loss 0.1766035

loss contrastive
tensor(0.1429, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1789, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1546, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1320, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1601, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1501, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1300, grad_fn=<MeanBackward1>)
Epoch number 1
 Current loss 0.13001485168933868

loss contrastive
tensor(0.1624, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1629, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1525, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1803, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1600, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1664, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1448, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1749, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1343, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1567, grad_f

loss contrastive
tensor(0.1527, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1764, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1630, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1597, grad_fn=<MeanBackward1>)
Epoch number 1
 Current loss 0.15968920290470123

loss contrastive
tensor(0.1371, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1320, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1420, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1596, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1292, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1526, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1089, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1649, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1899, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1587, grad_fn=<MeanBackward1>)
Epoch number 1
 Current loss 0.15871106088161469

loss contrastive
tensor(0.1233, grad_fn=<MeanBackward1>)
loss contrastive
tensor(0.1157, grad_fn=<Mean

In [57]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [334]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    batch_num = 0
    with torch.no_grad():
        for i, data in enumerate(iterator,0):
            img0, img1 , label = data
            output1,output2 = model(img0,img1)
            euclidean_distance = F.pairwise_distance(output1, output2)
            print(euclidean_distance)
            print(label)
            print("*********")

In [335]:
# Evaluate model performance
evaluate(model_CNN, val_batch_it, criterion)

tensor([1.0938e-01, 2.6486e-02, 1.4764e-02, 3.8404e-02, 6.3206e-03, 7.0899e-03,
        1.2306e-01, 5.6804e-02, 3.7451e-02, 5.7852e-02, 3.9487e-02, 2.5692e-02,
        1.7941e-02, 3.4291e-02, 2.7424e-02, 1.2709e-02, 6.4257e-02, 2.8552e-02,
        5.6959e-02, 2.6519e-02, 8.4709e-02, 7.3856e-02, 5.2609e-02, 1.0000e-06,
        1.3570e-02, 1.9449e-02, 2.0647e-03, 8.6965e-01, 8.1846e-01, 7.8720e-01,
        8.7804e-01, 8.4439e-01, 7.2466e-01, 8.8908e-01, 8.2758e-01, 8.7456e-01,
        8.4079e-01, 8.2444e-01, 8.4884e-01, 8.6154e-01, 8.4823e-01, 7.2655e-01,
        8.2865e-01, 8.6334e-01, 7.6907e-01, 7.8102e-03, 8.7255e-01, 8.1188e-01,
        7.8065e-01, 8.1480e-01, 7.7014e-01, 8.5584e-01, 8.3960e-01, 7.9126e-01,
        8.3739e-01, 1.7014e-02, 3.6031e-02, 1.8702e-02, 7.7624e-01, 3.3455e-02,
        8.7863e-01, 8.7564e-01, 8.6967e-01, 8.0924e-01, 8.5961e-01, 8.2029e-01,
        8.2883e-01, 8.9241e-01, 8.7226e-01, 8.5741e-01, 5.9573e-01, 8.9102e-01,
        7.3306e-01, 8.3024e-01, 7.9009e-

tensor([0.0956, 0.8948, 0.9154, 0.8692, 0.0508, 0.8761, 0.8545, 0.0696, 0.1075,
        0.8704, 0.8773, 0.0286, 0.0985, 0.1070, 0.0522, 0.0881, 0.6787, 0.0921,
        0.4647, 0.4445, 0.3586, 0.3602, 0.4600, 0.4470, 0.4815, 0.4409, 0.8481,
        0.8393, 0.3997, 0.4380, 0.3932, 0.8782, 0.8710, 0.8869, 0.8746, 0.8901,
        0.9025, 0.6913, 0.8760, 0.8701, 0.8241, 0.8368, 0.8610, 0.7903, 0.8219,
        0.7871, 0.8568, 0.8423, 0.8622, 0.8214, 0.8121, 0.8375, 0.7527, 0.8615,
        0.8359, 0.8435, 0.8305, 0.8636, 0.8069, 0.7672, 0.8257, 0.8697, 0.8501,
        0.7716, 0.8470, 0.8442, 0.7432, 0.8132, 0.8211, 0.8616, 0.8517, 0.8007,
        0.8121, 0.9104, 0.7722, 0.8640, 0.9088, 0.6726, 0.7595, 0.7977, 0.8973,
        0.8967, 0.8050, 0.8701, 0.9091, 0.9197, 0.9084, 0.8929, 0.7332, 0.8988,
        0.9183, 0.9345, 0.6679, 0.9263, 0.1572, 0.9204, 0.7939, 0.5095, 0.5274,
        0.5036, 0.4925, 0.4859, 0.4364, 0.5095, 0.3596, 0.4890, 0.1840, 0.2465,
        0.2411, 0.1942, 0.2136, 0.7497, 

tensor([0.7308, 0.7999, 0.7484, 0.7937, 0.8312, 0.7870, 0.7508, 0.7527, 0.6923,
        0.7470, 0.8056, 0.7436, 0.7936, 0.8008, 0.7982, 0.6251, 0.8156, 0.7429,
        0.7889, 0.7224, 0.7296, 0.6567, 0.7297, 0.7626, 0.8375, 0.7629, 0.7496,
        0.8020, 0.6563, 0.7948, 0.7407, 0.8249, 0.7936, 0.8146, 0.7532, 0.8117,
        0.8261, 0.8018, 0.7842, 0.8431, 0.8041, 0.8311, 0.7409, 0.7410, 0.8298,
        0.8627, 0.8958, 0.8761, 0.8753, 0.8370, 0.8577, 0.8487, 0.7632, 0.8525,
        0.8765, 0.8180, 0.6944, 0.8381, 0.5674, 0.8578, 0.8444, 0.7560, 0.8609,
        0.8214, 0.7854, 0.6328, 0.7756, 0.7622, 0.8434, 0.8417, 0.7428, 0.7535,
        0.7839, 0.5674, 0.7809, 0.8639, 0.7292, 0.8384, 0.8484, 0.7400, 0.8318,
        0.8019, 0.8119, 0.8273, 0.4887, 0.6270, 0.6217, 0.8106, 0.7135, 0.8505,
        0.5997, 0.8491, 0.8441, 0.8101, 0.8526, 0.8407, 0.6048, 0.8647, 0.4124,
        0.8549, 0.8139, 0.6210, 0.8153, 0.5342, 0.6388, 0.7847, 0.8333, 0.8136,
        0.7854, 0.4890, 0.8663, 0.5802, 

tensor([0.6081, 0.8278, 0.9020, 0.8923, 0.5396, 0.2639, 0.8926, 0.8551, 0.9096,
        0.9296, 0.9257, 0.5793, 0.7558, 0.5876, 0.1271, 0.8333, 0.5677, 0.6498,
        0.8680, 0.8996, 0.8794, 0.9383, 0.9295, 0.6389, 0.9312, 0.9010, 0.6162,
        0.8503, 0.8249, 0.7849, 0.7816, 0.4034, 0.6079, 0.8113, 0.6033, 0.8735,
        0.7984, 0.6046, 0.8492, 0.8253, 0.7849, 0.8129, 0.8582, 0.7408, 0.8108,
        0.8270, 0.8034, 0.5065, 0.8413, 0.8656, 0.8380, 0.5536, 0.8317, 0.3953,
        0.8043, 0.7999, 0.8246, 0.8248, 0.8525, 0.7710, 0.8122, 0.8023, 0.8333,
        0.8294, 0.5228, 0.8374, 0.7549, 0.7772, 0.8324, 0.7815, 0.7987, 0.7715,
        0.3976, 0.8771, 0.4384, 0.8368, 0.8158, 0.8254, 0.8732, 0.8586, 0.7979,
        0.8273, 0.8561, 0.8634, 0.8277, 0.3692, 0.0712, 0.8647, 0.8703, 0.4122,
        0.4237, 0.7207, 0.5516, 0.5936, 0.6089, 0.6444, 0.6394, 0.8319, 0.5754,
        0.7911, 0.6222, 0.6042, 0.6458, 0.6343, 0.7704, 0.6209, 0.5061, 0.6333,
        0.7677, 0.8803, 0.8688, 0.8899, 

KeyboardInterrupt: 

In [333]:
N_EPOCHS = 2
print(train_iter)
print(len(train_iter))
print(val_iter)
print(len(val_iter))
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train(model_CNN, train_iter, optimizer, criterion)
    print(train_loss)
    print(train_acc)
    valid_loss, valid_acc = evaluate(model_RNN, val_iter, criterion)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

293
98


TypeError: 'DataFrame' object is not callable